# Limpieza y Depuración de los Datos

En este notebook se realiza la lectura del dataset generado por scraping y se complementa la información del tipo de propiedad y municipios, así mismo se mejora la estructura del dataset (nombre de columnas y tipos de dato) para su posterior uso en el modelo de la iteación 1.

In [18]:
!pip install pandas==1.2.4

     |████████████████████████████████| 9.9MB 4.3MB/s 
ERROR: google-colab 1.0.0 has requirement pandas~=1.1.0; python_version >= "3.0", but you'll have pandas 1.2.4 which is incompatible.
  Found existing installation: pandas 1.1.5
    Uninstalling pandas-1.1.5:
      Successfully uninstalled pandas-1.1.5


In [1]:
!pip install unidecode

In [2]:
import time
import numpy as np
import pandas as pd

In [3]:
#Descargar datasets desde github
!git clone https://github.com/andres-soto-h/monografia-udea-eacd.git

fatal: destination path 'monografia-udea-eacd' already exists and is not an empty directory.


### Lectura de datos y cruce de información

In [35]:
df_propiedades=pd.read_csv('/content/monografia-udea-eacd/consolidado_propiedades_FR_30052021.csv', delimiter=';', encoding='latin1') 
print(df_propiedades.shape)

df_muni = pd.read_csv('/content/monografia-udea-eacd/davipola.csv', delimiter=',')
df_muni.shape


(2715, 188)


(1126, 5)

In [36]:
# Aplicar limpieza a los nombres de las columnas para quitar caracteres extraños y evitar errores
lista_columnas=list(df_propiedades.columns)
col_limpia=[]

import unidecode

for columna in lista_columnas:
  col=columna.lower().replace('/','').replace('.','').strip().replace('  ',' ').replace('  ',' ').replace(' ', '_')
  col=unidecode.unidecode(col)
  col_limpia.append(col)

columnas_nuevas=dict(zip(lista_columnas,col_limpia))

In [37]:
df_propiedades.rename(columns=columnas_nuevas, inplace=True)
df_propiedades.rename(columns={'area':'area_m2'}, inplace=True)

In [38]:
df_propiedades.head(1)

,tipo,url,titulo,precio,area_m2,habitaciones,banos,garajes,descripcion,balcon,bano_de_servicio,chimenea,cocina_equipada,cuarto_de_servicio,sauna_turco_jacuzzi,servicio_de_internet,servicios_publicos,zona_de_bbq,acceso_pavimentado,garaje_parqueadero(s),porteria_vigilancia,senderos_ecologicos,area_rural,cerca_de_zona_urbana,sobre_via_secundaria,trans_publico_cercano,caractint,caractext,caractsec,estrato,antiguedad,cocina_integral,ventilacion_natural,asador,bosque_nativo,establo,invernadero,pesebrera,terraza,vista_panoramica,...,puerta_electrica,ubicada_en_edificio,finca_agricola,sensor_de_movimiento,planta_electrica,servicios_independientes,cuarto_de_escoltas,salon_de_conferencias,rociadores_de_agua,con_casa_club,disponibilidad_wifi,galpon,piso_de_alta_resistencia,cableado_de_red,puerta_de_seguridad,sala_de_internet,finca_avicola,lote_en_construccion,salon_de_videoconferencias,pasaje_comercial,sistema_de_riego,acceso_para_camiones,con_vivienda,lic_de_construccion,panoramica_un_lado,en_club,auditorio,esquinero,oficinas_administrativas,con_casa_prefabricada,en_zona_comercial,finca_cafetera,servicio_de_lavanderia,altura_restringida,ascensor(es)_inteligente(s),lote_vacio,control_de_acustica,control_termico,panoramica_360o,con_cerca_electrica
0,Usada,https://www.fincaraiz.com.co/casa-campestre-en...,Casa Campestre en Venta Rionegro Vereda Guayabito,$ 1.380.000.000,"320,00 m²",5,6,2,Propiedad moderna con muy buenos acabados. Her...,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,"['Balcón', 'Baño de Servicio', 'Chimenea', 'Co...","['Acceso Pavimentado', 'Garaje / Parqueadero(s...","['Área Rural', 'Cerca de Zona Urbana', 'Sobre ...",5,1 a 8 años,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
df_muni_antioquia=df_muni[df_muni["Nombre Departamento"]=='ANTIOQUIA']
df_muni_antioquia.head(1)

,Código Departamento,Código Municipio,Nombre Departamento,Nombre Municipio,Tipo: Municipio / Isla / Área no municipalizada
0,05,5001,ANTIOQUIA,MEDELLÍN,Municipio


In [40]:
#Aplicar transformación a los datos de municipios para usar el nombre como id de referencia en ambos datasets
df_muni_antioquia["Nombre Municipio"]=df_muni_antioquia["Nombre Municipio"].apply(lambda x: unidecode.unidecode(x.lower()))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [41]:
list_muni=np.array(df_muni_antioquia["Nombre Municipio"])
list_muni=np.append(list_muni, np.array(['ceja','viboral','llano','llanogrande','pereira','union','unión','santuario']))

In [42]:
#Funcion para extraer la ubicación de la propiedad a partir de los campos del dataset
def extract_ubicacion(lineas):
  texto=np.array(lineas)[0]
  palabras=texto.lower().replace(',','').replace(';','').split(' ')
  muni=''
  
  for palabra in palabras:

    if palabra in list_muni:
        muni = palabra
        break
    else:
        muni = ''

  if muni == '':
    texto=np.array(lineas)[1]
    palabras=texto.lower().replace(',','').replace(';','').split(' ')
  
    for palabra in palabras:

      if palabra in list_muni:
          muni = palabra
          break
      else:

          muni = ''


  return muni

In [43]:
#Funcion para extraer el tipo de propiedad a partir de los campos del dataset
tipo_propiedades=['casa','finca','apartamento','apartaestudio','lote', 'apartamentos']

def extract_tipo(lineas):

  texto=np.array(lineas)[0]

  palabras=texto.lower().split(' ')
  tipo_prop=''
  
  for palabra in palabras:

    if palabra in tipo_propiedades:
        tipo_prop = palabra
        break
    else:
        tipo_prop = ''

  if tipo_prop == '':

    texto=np.array(lineas)[1]

    palabras=texto.lower().split(' ')
    tipo_prop=''
    
    for palabra in palabras:

      if palabra in tipo_propiedades:
          tipo_prop = palabra
          break
      else:
          #tipo de propiedad por defecto será apartamento
          tipo_prop = 'apartamento'

  return tipo_prop

In [44]:
df_propiedades["municipio"]=df_propiedades[["titulo","descripcion"]].apply(extract_ubicacion,axis=1)

In [45]:
df_propiedades["tipo_propiedad"]=df_propiedades[["titulo","descripcion"]].apply(extract_tipo, axis=1)

In [46]:
df_propiedades.describe(include=['O'])

,tipo,url,titulo,precio,area_m2,habitaciones,banos,garajes,descripcion,caractint,caractext,caractsec,estrato,antiguedad,municipio,tipo_propiedad
count,2715,2715,2715,2715,2715,2715,2715,2715,2715,2715,2715,2715,2635,2188,2715,2715
unique,2,2715,1297,411,667,17,15,12,2624,1862,1678,578,7,6,11,5
top,Usada,https://www.fincaraiz.com.co/casa-campestre-en...,Apartamento en Venta Rionegro RIONEGRO,$ 320.000.000,"200,00 m²",3,2,1,6351568 Proyecto de apartamentos de 2 y 3 habi...,[],[],[],4,1 a 8 años,rionegro,casa
freq,2678,1,79,51,54,1153,1032,1200,10,302,426,450,1014,1167,1360,1189


In [47]:
df_propiedades.info(show_counts=True, max_cols=190)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2715 entries, 0 to 2714
Data columns (total 190 columns):
 #    Column                       Non-Null Count  Dtype  
---   ------                       --------------  -----  
 0    tipo                         2715 non-null   object 
 1    url                          2715 non-null   object 
 2    titulo                       2715 non-null   object 
 3    precio                       2715 non-null   object 
 4    area_m2                      2715 non-null   object 
 5    habitaciones                 2715 non-null   object 
 6    banos                        2715 non-null   object 
 7    garajes                      2715 non-null   object 
 8    descripcion                  2715 non-null   object 
 9    balcon                       1401 non-null   float64
 10   bano_de_servicio             201 non-null    float64
 11   chimenea                     459 non-null    float64
 12   cocina_equipada              378 non-null    float64
 13   c

### Realizar limpieza de datos y llenar nulos

In [48]:
df_propiedades['municipio'].value_counts()

rionegro       1360
retiro          474
ceja            368
guarne          212
viboral          93
marinilla        90
llano            53
pereira          43
santuario        13
llanogrande       5
unión             4
Name: municipio, dtype: int64

In [49]:
df_propiedades['tipo_propiedad'].value_counts()

casa             1189
apartamento       915
finca             564
apartaestudio      31
apartamentos       16
Name: tipo_propiedad, dtype: int64

In [50]:
df_propiedades['tipo'].value_counts()

Usada    2678
Nueva      37
Name: tipo, dtype: int64

In [51]:
list(df_propiedades[df_propiedades['tipo_propiedad']==""]["url"])

[]

In [52]:
#unificar apartamentos
df_propiedades['tipo_propiedad']=df_propiedades['tipo_propiedad'].apply(lambda x: x.replace("apartamentos", "apartamento"))


In [53]:
df_propiedades['tipo_propiedad'].value_counts()

casa             1189
apartamento       931
finca             564
apartaestudio      31
Name: tipo_propiedad, dtype: int64

In [54]:
def unifica_muni(texto):
  muni=''
  
  if texto=='retiro':
    muni="el retiro"

  elif texto=='ceja':
    muni="la ceja"
 
  elif texto=='viboral':
    muni="el carmen de viboral"
  
  elif texto=='llano':
    muni="llanogrande"

  elif texto=='unión':
    muni="la unión"

  elif texto=='pereira':
    muni="san antonio de pereira"

  else:
    return texto

  return muni

In [55]:
df_propiedades['municipio']=df_propiedades['municipio'].apply(unifica_muni)

In [56]:
df_propiedades['municipio'].value_counts()

rionegro                  1360
el retiro                  474
la ceja                    368
guarne                     212
el carmen de viboral        93
marinilla                   90
llanogrande                 58
san antonio de pereira      43
santuario                   13
la unión                     4
Name: municipio, dtype: int64

In [57]:
#Llenar valores nulos con cero
df_propiedades=df_propiedades.fillna(0)

In [58]:
pd.set_option('display.float_format', lambda x: '%.0f' % x)

In [59]:
#Transformar variable precio
df_propiedades['precio']=df_propiedades['precio'].apply(lambda x: x.replace('$','').replace('.','').replace(',','.').strip()).astype(float)

In [60]:
#transformar variable área
df_propiedades['valida_ha']=df_propiedades['area_m2'].apply(lambda x: 1 if x.find("Ha")!=-1 else 0)

In [61]:
#Limpiar texto y convertir a número
df_propiedades['area_m2'] = df_propiedades['area_m2'].apply(lambda x: x.replace('Ha','').replace('m2','').replace('m²','').replace('.','').replace(',','.').strip()).astype(float)

In [62]:
#Corregir unidades hectareas
df_propiedades['area_m2'] = df_propiedades[['area_m2','valida_ha']].apply(lambda x: x.area_m2 if x.valida_ha == 0 else x.area_m2*10000,axis=1)

In [63]:
df_propiedades.drop(['valida_ha'], axis=1, inplace=True)

In [64]:
#Transformar variable habitaciones -- > El número por defecto será 1 sino especifica
df_propiedades['habitaciones']=df_propiedades['habitaciones'].apply(lambda x: x.replace('---','1').replace('Sin especificar','1').strip()).astype(float)

In [65]:
#Transformar variable baños -- > El número por defecto será 1 sino especifica
df_propiedades['banos']=df_propiedades['banos'].apply(lambda x: x.replace('---','1').replace('Sin especificar','1').strip()).astype(float)

In [66]:
#Transformar variable garajes -- > El número por defecto será 0 sino especifica
df_propiedades['garajes']=df_propiedades['garajes'].apply(lambda x: x.replace('---','0').replace('Sin especificar','0').replace('Más de 10','10').strip()).astype(float)

In [67]:
df_propiedades['habitaciones'].value_counts()

3     1153
4      639
2      439
5      248
1      129
6       62
7       26
8        9
9        4
15       1
21       1
12       1
20       1
10       1
14       1
Name: habitaciones, dtype: int64

In [68]:
df_propiedades.info(show_counts=True, max_cols=190)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2715 entries, 0 to 2714
Data columns (total 190 columns):
 #    Column                       Non-Null Count  Dtype  
---   ------                       --------------  -----  
 0    tipo                         2715 non-null   object 
 1    url                          2715 non-null   object 
 2    titulo                       2715 non-null   object 
 3    precio                       2715 non-null   float64
 4    area_m2                      2715 non-null   float64
 5    habitaciones                 2715 non-null   float64
 6    banos                        2715 non-null   float64
 7    garajes                      2715 non-null   float64
 8    descripcion                  2715 non-null   object 
 9    balcon                       2715 non-null   float64
 10   bano_de_servicio             2715 non-null   float64
 11   chimenea                     2715 non-null   float64
 12   cocina_equipada              2715 non-null   float64
 13   c

In [69]:
# Eliminar del dataset propiedades con precios erroneos o posibles outliers
df_propiedades=df_propiedades[df_propiedades["precio"]<=2500000000]

In [70]:
df_propiedades.shape

(2478, 190)

In [71]:
df_propiedades.to_csv('df_prop_clean.csv', header=True, index=False, sep=";")